In [10]:
import pathlib
from datetime import datetime

from astropy.table import Table
from astropy.table import Column
from astropy.time import Time
import astropy.coordinates as coord
import astropy.units as u


import numpy as np

# Set up matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

%matplotlib inline

# See https://mpld3.github.io/quickstart.html !!

# or https://docs.bokeh.org/en/latest/
# https://docs.bokeh.org/en/latest/docs/user_guide/quickstart.html#userguide-quickstart


from bokeh.plotting import figure, output_file, output_notebook, show, save
from bokeh.models import Legend, ColumnDataSource, HoverTool
from bokeh.layouts import row, column

import pandas

In [2]:
# output to static HTML file (with CDN resources)
output_file("bokeh_plot2.html", title="Test Bokeh Plot", mode="cdn")
#output_notebook()


In [3]:
data_dir = pathlib.Path('/Users/donald/Box Sync/navo_stats')
#file = 'ps_cone_1000_2019-04-14.csv'
file = 'ps_cone-2019-07-15-23:37.csv'
path = data_dir / file

# Read csv into Table
old_data = Table.read(path, format='ascii.csv')
data = old_data

In [4]:
data_dir = pathlib.Path('/Users/donald/Box Sync/navo_stats')
file = 'ps_cone-2019-11-12-13-58.csv'
path = data_dir / file

# Read csv into Table
medium_data = Table.read(path, format='ascii.csv')
data = medium_data

In [12]:
def qt_vs_num_rows():
    # create a new plot with a datetime axis type
    p = figure(plot_width=500, plot_height=500)

    data["num_rows"] = data["num_rows"].astype(float)  # Gets rid of the warning when implicitly converted for Pandas
    df = data["int0_duration", "int1_duration", "num_rows", "base_name", "service_type",
              "RA", "DEC", "SR"].to_pandas().copy()
    source = ColumnDataSource(df)

    # add renderers
    qt_rend = p.circle(x="int0_duration", y="num_rows", source=source, size=4, color='red', alpha=0.2)
    dt_rend = p.circle(x="int1_duration", y="num_rows", source=source, size=4, color='green', alpha=0.2)

    legend = Legend(items=[
        ("Query Time",   [qt_rend]),
        ("Download Time", [dt_rend])
    ], location=(0, 40), click_policy='hide')
    p.add_layout(legend, 'below')

    hover = HoverTool(
        tooltips=[
            ("RA", "@RA"),
            ("Dec", "@DEC"),
            ("Radius", "@SR"),
            ("Query Time", "@int0_duration"),
            ("Download Time", "@int1_duration"),
            ("# of Rows", "@num_rows"),
            ("Start Time", "@dt_start_time{%Y-%m-%d-%H:%M:%S}")
        ],
        formatters={
            'dt_start_time': 'datetime'
        }
    )
    p.add_tools(hover)

    # NEW: customize by setting attributes
    p.title.text = "Times versus # of Result Rows"
    #p.grid.grid_line_alpha = 0
    p.xaxis.axis_label = 'Duration'
    p.yaxis.axis_label = '# of Rows'
    #p.ygrid.band_fill_color = "olive"
    #p.ygrid.band_fill_alpha = 0.05


    # show the results
    # show(p)
    # save(p)
    
    return p

# Todo
# - Put base name and service type in title
# - Put start time rows and duration in tooltip
# - Do similar thing for vs. radius plot
# - Do time plot
# - Put them all in a web page


In [13]:
def qt_over_time():

    # create a new plot with a datetime axis type
    p = figure(plot_width=700, plot_height=500, x_axis_type="datetime")

    data["num_rows"] = data["num_rows"].astype(float)  # Gets rid of the warning when implicitly converted for Pandas
    df = data["start_time", "int0_duration", "int1_duration", "num_rows", "base_name", "service_type",
              "RA", "DEC", "SR"].to_pandas().copy()
    df["dt_start_time"] = pandas.to_datetime(df["start_time"], format='%Y-%m-%d-%H:%M:%S.%f')

    source = ColumnDataSource(df)

    # add renderers
    qt_rend = p.circle(x="dt_start_time", y="int0_duration", source=source, size=4, color='red', alpha=0.2)

    hover = HoverTool(
        tooltips=[
            ("RA", "@RA"),
            ("Dec", "@DEC"),
            ("Radius", "@SR"),
            ("Query Time", "@int0_duration"),
            ("Download Time", "@int1_duration"),
            ("# of Rows", "@num_rows"),
            ("Start Time", "@dt_start_time{%Y-%m-%d-%H:%M:%S}")
        ],
        formatters={
            'dt_start_time': 'datetime'
        }
    )
    p.add_tools(hover)

    # NEW: customize by setting attributes
    p.title.text = "Query Time Over Time"
    p.xaxis.axis_label = 'Start Time'
    p.yaxis.axis_label = 'Query Time (s)'


    # show the results
    # save(p)
    
    return p

In [17]:
qt_vs_num_rows_plot = qt_vs_num_rows()
qt_over_time_plot = qt_over_time()

save((qt_vs_num_rows_plot, qt_over_time_plot))

'/Users/donald/dev/astropy/basic_plots/sandbox/bokeh_plot2.html'